# Feature Engineering Notebook

## Objectives
* 
*

## Inputs

*

## Outputs

*

## Conclusions
*

---

## Change working directory

We need to change the working directory from its current folder to its parent folder

* We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

We want to make the parent of the current directory the new current directory

* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

Confirm the new directory

In [ ]:
current_dir = os.getcwd()
current_dir

## Load Data

In [ ]:
import pandas as pd
df_house_prices = pd.read_csv(f"outputs/datasets/collection/house_prices_records.csv")
df_house_prices.head()